In [4]:
## time and date of the scan --> date of scanning  must be greater than the last scan submission date
##'completed' tag must be present
## hostname should match with the file name
## RiskFound must be 0

import pytesseract
pytesseract.pytesseract.tesseract_cmd ='D:/Tesseract-OCR/tesseract'
%pylab inline
import cv2

Populating the interactive namespace from numpy and matplotlib


In [39]:
import re

In [84]:
def helper_function(res,lst):
    txt =pytesseract.image_to_string(res)
    
    indices = [i for i, x in enumerate(cd) if x == "xx"]
    
    
    
    ##preprocessing
    
    result1 = re.sub(r"\n", " ", txt, flags=re.I) 
    result2 = re.sub(r"\\", "", result1)
    result3 = re.sub(r"|", "", result2)
    txt = re.sub(r"¢", "0", result3)
    
    
    ##date
    try:
        if 0 in set(indices):
            lst[0] = 'Date: '+re.search(r'(\d+(/|-){1}\d+(/|-){1}\d{2,4})', txt)[0]
    except:
        pass
    
    
    #time
    try:
        if 0 in set(indices):
            lst[1] = "Time:"+re.search(r'(\d+:\d+:\d+)', txt)[0]
    except:
        pass

    
    ##scan completed or not
    try:
        if (2 in set(indices)):
            if re.findall('Completed', txt):
                lst[2] = 'completed'
            else:
                lst[2]='error'
    except:
        pass

    
    ##risk
    try:
        if 3 in set(indices):
            lst[3] = re.search(r"Risks found:(?:[^a-zA-Z'-]+){1}", txt)[0]
        
    except:
        pass


    ##files scanned
    try:
        if 4 in set(indices):
            lst[4] = re.search(r"scanned:(?:[^a-zA-Z'-]+){1}", txt)[0]
    except:
        pass

    
    ##hostname
    try:
        if 5 in set(indices):
            lst[5] = re.search(r"KIIT_(?:[^a-zA-Z'-]){5}", txt)[0][5:]
    except:
        pass
    
    return lst
    

In [85]:
def data_retriver(img):
    lst = ['xx','xx','xx','xx','xx','xx']
    ##2XSCALING
    scale_percent = 200 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    res = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    if 'xx' in set(lst):
        ret,thresh2 = cv2.threshold(res,127,255,cv2.THRESH_BINARY_INV)
        lst = helper_function(thresh2,lst)
    else:
        return lst

    if 'xx' in set(lst):
        rest = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        th1 = cv2.adaptiveThreshold(rest,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
        lst = helper_function(th1,lst)
    else:
        return lst    
    
    ##LAB
    lab= cv2.cvtColor(res, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(10,10))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    lst = helper_function(limg,lst)
    
    
    


    
    
    return lst
         

    
    

In [86]:
%%time
img = cv2.imread("we7.png")

lst = data_retriver(img)
lst

Wall time: 7.89 s


In [87]:
lst

['Date: 8/21/2019',
 'Time:2:52:30',
 'completed',
 'Risks found: 0 | ',
 'scanned: 2,945,432  ',
 '10471']